# Chapter 1: The Basics of Prompting

**Scenario:** You are planning a weekend trip to **Napa Valley**. You have a budget, specific dates, and interests. You need the AI to act as your **Travel Agent** to create the perfect itinerary.

In this notebook, you will learn:
- 🔧 **Setup**: How to connect to the AI model.
- 🗣️ **The Messages Format**: How to speak to the AI (User vs. Assistant).
- 🏗️ **Core Structure**: The 3 key parts of a prompt: **Context**, **Task**, and **Format**.

---

## 1. Setup

First, we need to get our tool ready. We use a library called `litellm` to talk to different AI models.

In [1]:
# Install tools (if needed)
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# 1. Hide messy tech logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

# 2. Load secret keys
load_dotenv()

# 3. Pick our model
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')
print(f"Ready to plan trips with: {MODEL_NAME}")

Ready to plan trips with: openrouter/google/gemini-2.0-flash-001


### Helper Function

This simple function sends your message to the AI and gets the answer. You don't need to understand the code inside, just how to use it.

In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. Basic Prompting

Let's try a very simple request (a "prompt").

In [4]:
print(get_completion("Plan a trip to Napa Valley.", temperature=0.7))

Okay, let's plan a trip to Napa Valley! I'll create a customizable itinerary, covering lodging, transportation, wineries, dining, and activities.  To make this plan perfect for you, I need a little more information:

**1.  Trip Duration:** How many days/nights will you be in Napa? (e.g., 3 days/2 nights, 4 days/3 nights, a week, etc.)

**2.  Time of Year:** When are you planning to travel? (Month/Season). This significantly impacts pricing, weather, and event availability.  Harvest season (August-October) is beautiful but crowded and more expensive.

**3.  Budget:**  Roughly, what's your budget per day (excluding flights)?  (e.g., Budget-friendly - $200-$300/day, Mid-range - $300-$500/day, Luxury - $500+/day). This will help determine lodging and dining options.

**4.  Travelers:** How many people are in your group? (This impacts lodging needs.)

**5.  Interests:** What are your priorities? (Check all that apply)
    *   Wine Tasting: (Specific varietals?  e.g., Cabernet Sauvignon focu

This answer is probably too general. The AI doesn't know what you like, your budget, or when you are going. It's like asking a stranger "Recommend a restaurant" without telling them what food you like.

## 3. The Structure of a Good Prompt

To get a great result, use this 3-part structure:

1.  **CONTEXT**: Background info (Dates, Budget, Who is going?).
2.  **TASK**: Exactly what you want the AI to do.
3.  **FORMAT**: How the answer should look (List, Email, Itinerary).

Let's rewrite our prompt using this structure.

In [5]:
prompt = """
CONTEXT: 
I am planning a weekend trip to Napa Valley. 
Duration: 2 days (Saturday-Sunday). 
Budget: $500 total.
Interests: Wine tasting, scenic views, and good food (Italian).

TASK: 
Create a detailed 2-day itinerary.

FORMAT: 
Structure the response by day (Saturday, Sunday) with:
- Morning Activity
- Lunch Recommendation
- Afternoon Activity
- Dinner Recommendation
- Estimated cost for each item
"""

print(get_completion(prompt, temperature=0.7, max_tokens=1024))

Okay, here's a possible Napa Valley itinerary focusing on wine tasting, scenic views, Italian food, and staying within a $500 budget for a 2-day weekend trip. Note that prices for wine tasting and food can vary considerably, so I've aimed for a mix of affordable and potentially slightly more upscale options. You can adjust based on your preferences and availability. Also, transportation costs (gas, ride-sharing) are not included within the budget to avoid underestimating these costs.

**Important Notes Before You Go:**

*   **Book in Advance:** Reservations are HIGHLY recommended for wine tastings, especially on weekends. Book well in advance.
*   **Transportation:** Napa is spread out. Consider ride-sharing services (Uber/Lyft), but be aware these can be expensive and availability might be limited. Designate a driver, or use a wine tour company if you plan to drink heavily. The Vine public bus is an option but may not be ideal for maximizing your time.
*   **Pace Yourself:** Wine tast

Reflect: Notice how much better the answer is? The AI knew exactly what constraints to follow.

## 4. System Prompts (The "Persona")

You can also give the AI a "System Prompt". This tells the AI **who it is** or **how it should behave** before it even sees your specific request.

Think of it like giving an actor their character before they read the script.

In [6]:
system_prompt = "You are an expert Luxury Travel Agent. You are polite, professional, and you love finding hidden gems."

user_prompt = "Plan a trip to Napa Valley."

print(get_completion(user_prompt, system_prompt, temperature=0.7, max_tokens=1024))

Okay! I'd love to help you plan a luxurious and memorable trip to Napa Valley. To craft the perfect itinerary, I need a little more information from you.

**First, tell me about your preferences:**

*   **What time of year are you planning to travel?** (This will greatly impact pricing, crowds, and activities.)
*   **How long will you be in Napa Valley?** (Knowing the number of days will help me structure the itinerary.)
*   **What is your budget for this trip?** (Knowing your budget will help me narrow down the options for accommodations, activities, and dining.)
*   **Who are you traveling with?** (Partner, friends, family, etc.)
*   **What are your interests beyond wine?** (Fine dining, spa treatments, hiking, art, shopping, hot air ballooning, history, etc.)
*   **What kind of wine do you enjoy?** (Cabernet Sauvignon, Chardonnay, Pinot Noir, sparkling wine, etc.)
*   **What is your preferred style of accommodation?** (Boutique hotel, luxury resort, private villa, charming inn, etc.

## 5. Exercise: Plan Your Own Event

Now it's your turn. Use the **Context-Task-Format** structure to plan a different event.

**Goal:** Plan a Birthday Party.
**Constraints:**
-   **Context**: 10 year old's birthday. 15 kids. Outdoors.
-   **Task**: Plan 3 games and a food menu.
-   **Format**: Bulleted list with time requirements for games.

In [7]:
my_prompt = """
CONTEXT:
[Your context here]

TASK:
[Your task here]

FORMAT:
[Your format here]
"""

# print(get_completion(my_prompt))

## Summary

1.  **Be specific**. Don't just ask for a "trip"; ask for a "2-day trip with a $500 budget".
2.  **Use Structure**. Separate your **Context**, **Task**, and **Format**.
3.  **Use personas**. Tell the AI who to be (System Prompt).